In [98]:
import pandas as pd
import plotly.express as px
import numpy as np
from datetime import datetime, timedelta
import json


In [ ]:


listening_df = pd.read_csv("datasets/user_clean/ReRe_20250602_164123.csv")
charts_df = pd.read_csv("datasets/info_clean/info_charts_weighted.csv")

def calculate_chart_listening_points(listening_df, charts_df, window_days=30):
    """
    Calculate points for listening to chart songs within a rolling window.

    Parameters:
    - listening_df: DataFrame with columns ['datetime', 'artist_name', 'track_name']
    - charts_df: DataFrame with columns ['weekdate', 'artist_name', 'track_name', 'position', 'weighting']
    - window_days: Rolling window in days (default 30 for ~1 month)

    Returns:
    - DataFrame with listening instances and awarded points
    """

    # Ensure datetime columns are properly formatted
    listening_df = listening_df.copy()
    charts_df = charts_df.copy()

    listening_df['datetime'] = pd.to_datetime(listening_df['datetime'])
    charts_df['weekdate'] = pd.to_datetime(charts_df['weekdate'])

    # Create a list to store results
    results = []

    # For each listening instance
    for idx, listen_row in listening_df.iterrows():
        listen_datetime = listen_row['datetime']
        artist = listen_row['artist_name']
        track = listen_row['track_name']

        # Define the rolling window (30 days before and after the listen date)
        window_start = listen_datetime - timedelta(days=window_days)
        window_end = listen_datetime + timedelta(days=window_days)

        # Find matching chart entries within the window
        chart_matches = charts_df[
            (charts_df['artist_name'] == artist) &
            (charts_df['track_name'] == track) &
            (charts_df['weekdate'] >= window_start) &
            (charts_df['weekdate'] <= window_end)
        ]

        # Calculate total points for this listening instance
        total_points = chart_matches['weighting'].sum() if not chart_matches.empty else 0

        # Store the result
        result_row = {
            'datetime': listen_datetime,
            'artist_name': artist,
            'track_name': track,
            'points_awarded': total_points,
            'chart_weeks_matched': len(chart_matches)
        }

        # Optional: Add details about which chart weeks were matched
        if not chart_matches.empty:
            result_row['chart_weeks_detail'] = chart_matches[['weekdate', 'position', 'weighting']].to_dict('records')
        else:
            result_row['chart_weeks_detail'] = []

        results.append(result_row)

    return pd.DataFrame(results)

def calculate_listener_summary(points_df):
    """
    Calculate summary statistics for the listener.

    Parameters:
    - points_df: DataFrame returned from calculate_chart_listening_points

    Returns:
    - Dictionary with summary statistics
    """

    total_listens = len(points_df)
    chart_listens = len(points_df[points_df['points_awarded'] > 0])
    total_points = points_df['points_awarded'].sum()
    avg_points_per_listen = points_df['points_awarded'].mean()

    # Points by time period
    points_df_copy = points_df.copy()
    points_df_copy['date'] = points_df_copy['datetime'].dt.date
    points_df_copy['week'] = points_df_copy['datetime'].dt.to_period('W')
    points_df_copy['month'] = points_df_copy['datetime'].dt.to_period('M')

    daily_points = points_df_copy.groupby('date')['points_awarded'].sum()
    weekly_points = points_df_copy.groupby('week')['points_awarded'].sum()
    monthly_points = points_df_copy.groupby('month')['points_awarded'].sum()

    return {
        'total_listening_instances': total_listens,
        'chart_song_listens': chart_listens,
        'chart_hit_rate': chart_listens / total_listens if total_listens > 0 else 0,
        'total_points': total_points,
        'average_points_per_listen': avg_points_per_listen,
        'best_day_points': daily_points.max() if not daily_points.empty else 0,
        'best_week_points': weekly_points.max() if not weekly_points.empty else 0,
        'best_month_points': monthly_points.max() if not monthly_points.empty else 0
    }

def optimize_for_large_datasets(listening_df, charts_df, window_days=30):
    """
    Optimized version for large datasets using vectorized operations.
    More memory efficient but requires sufficient RAM.
    """

    # Prepare data
    listening_df = listening_df.copy()
    charts_df = charts_df.copy()

    listening_df['datetime'] = pd.to_datetime(listening_df['datetime'])
    charts_df['weekdate'] = pd.to_datetime(charts_df['weekdate'])

    # Create composite keys for faster matching
    listening_df['song_key'] = listening_df['artist_name'] + '|||' + listening_df['track_name']
    charts_df['song_key'] = charts_df['artist_name'] + '|||' + charts_df['track_name']

    # Pre-filter charts to only songs that appear in listening history
    relevant_songs = set(listening_df['song_key'].unique())
    charts_filtered = charts_df[charts_df['song_key'].isin(relevant_songs)].copy()

    results = []

    # Group charts by song for faster lookup
    charts_grouped = charts_filtered.groupby('song_key')

    for song_key in relevant_songs:
        if song_key not in charts_grouped.groups:
            continue

        song_charts = charts_grouped.get_group(song_key)
        song_listens = listening_df[listening_df['song_key'] == song_key]

        for _, listen_row in song_listens.iterrows():
            listen_datetime = listen_row['datetime']

            # Calculate window
            window_start = listen_datetime - timedelta(days=window_days)
            window_end = listen_datetime + timedelta(days=window_days)

            # Find matching chart weeks
            matching_weeks = song_charts[
                (song_charts['weekdate'] >= window_start) &
                (song_charts['weekdate'] <= window_end)
            ]

            total_points = matching_weeks['weighting'].sum() if not matching_weeks.empty else 0

            results.append({
                'datetime': listen_datetime,
                'artist_name': listen_row['artist_name'],
                'track_name': listen_row['track_name'],
                'points_awarded': total_points,
                'chart_weeks_matched': len(matching_weeks)
            })

    return pd.DataFrame(results)

# Example usage and data preparation functions
# def prepare_sample_data():
    """
    Create sample data for testing the functions.
    """

    # Sample listening data
    listening_data = {
        'datetime': [
            '2024-01-15 14:30:25.123',
            '2024-01-16 09:15:30.456',
            '2024-01-20 20:45:10.789',
            '2024-02-01 11:20:05.234',
            '2024-02-15 16:35:45.567'
        ],
        'artist_name': ['Taylor Swift', 'Ed Sheeran', 'Billie Eilish', 'Taylor Swift', 'Drake'],
        'track_name': ['Anti-Hero', 'Shape of You', 'Bad Guy', 'Shake It Off', 'God\'s Plan']
    }

    # Sample chart data
    chart_data = {
        'weekdate': [
            '2024-01-08', '2024-01-15', '2024-01-22',
            '2024-01-08', '2024-01-15', '2024-01-22',
            '2024-02-05', '2024-02-12', '2024-02-19'
        ],
        'artist_name': [
            'Taylor Swift', 'Taylor Swift', 'Taylor Swift',
            'Ed Sheeran', 'Ed Sheeran', 'Billie Eilish',
            'Taylor Swift', 'Drake', 'Drake'
        ],
        'track_name': [
            'Anti-Hero', 'Anti-Hero', 'Anti-Hero',
            'Shape of You', 'Shape of You', 'Bad Guy',
            'Shake It Off', 'God\'s Plan', 'God\'s Plan'
        ],
        'position': [1, 2, 5, 10, 15, 8, 25, 3, 7],
        'weighting': [50, 49, 46, 41, 36, 43, 26, 48, 44]
    }

    listening_df = pd.DataFrame(listening_data)
    charts_df = pd.DataFrame(chart_data)

    return listening_df, charts_df

# Main execution example
# if __name__ == "__main__":
    # Load your data (replace with your actual data loading)
    # listening_df = pd.read_csv('your_listening_data.csv')
    # charts_df = pd.read_csv('your_chart_data.csv')

    # For demonstration, use sample data
    listening_df, charts_df = prepare_sample_data()

    print("Sample Listening Data:")
    print(listening_df.head())
    print("\nSample Chart Data:")
    print(charts_df.head())

    # Calculate points
    points_result = calculate_chart_listening_points(listening_df, charts_df, window_days=30)

    print("\nPoints Calculation Results:")
    print(points_result[['datetime', 'artist_name', 'track_name', 'points_awarded', 'chart_weeks_matched']])

    # Get summary statistics
    summary = calculate_listener_summary(points_result)

    print("\nListener Summary:")
    for key, value in summary.items():
        if isinstance(value, float):
            print(f"{key}: {value:.2f}")
        else:
            print(f"{key}: {value}")

    # Show detailed breakdown for chart hits
    chart_hits = points_result[points_result['points_awarded'] > 0]
    if not chart_hits.empty:
        print(f"\nDetailed Chart Hits ({len(chart_hits)} songs):")
        for idx, row in chart_hits.iterrows():
            print(f"\n{row['datetime'].strftime('%Y-%m-%d %H:%M')}: {row['artist_name']} - {row['track_name']}")
            print(f"  Points awarded: {row['points_awarded']}")
            print(f"  Chart weeks matched: {row['chart_weeks_matched']}")

In [14]:
results

NameError: name 'results' is not defined

In [36]:
charts_df = pd.read_csv("datasets/info_rawish/charts_raw.csv")
charts_df["weighting"] = 51 - charts_df["position"]
charts_df = charts_df.rename(columns={'artist': 'artist_name'})
charts_df = charts_df.rename(columns={'track': 'track_name'})
charts_df.to_csv("datasets/info_clean/info_charts_weighted.csv")


In [ ]:
pd.to_datetime(charts_df["weekdate"]).dtype

dtype('<M8[ns]')

In [47]:
charts_df = pd.read_csv("datasets/info_clean/info_charts_weighted.csv")
charts_df["weekdate"] = pd.to_datetime(charts_df["weekdate"], errors="raise")
charts_df.to_csv("datasets/info_clean/info_charts_weighted.csv")

In [49]:
charts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40150 entries, 0 to 40149
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Unnamed: 0   40150 non-null  int64         
 1   weekdate     40150 non-null  datetime64[ns]
 2   position     40150 non-null  int64         
 3   artist_name  40150 non-null  object        
 4   track_name   40150 non-null  object        
 5   weighting    40150 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 1.8+ MB


In [ ]:
elif page == "Charlies Play Place":
    # Show current user info
    user_selected = get_current_user(users)
    st.info(f"📊 Showing data for: **{user_selected}** (change user on Home page)")

    # Page title
    st.markdown("<h1 style='text-align: center; color: #32CD32;'>Spotify Regifted</h1>", unsafe_allow_html=True)

    # Load data
    listening_df = pd.read_csv("datasets/user_clean/ReRe_20250602_164123.csv")
    charts_df = pd.read_csv("datasets/info_clean/info_charts_weighted.csv")

    # Convert datetime columns
    listening_df['datetime'] = pd.to_datetime(listening_df['datetime'])
    charts_df['weekdate'] = pd.to_datetime(charts_df['weekdate'])

    window_days = 30
    results = []

    # Calculate chart points
    for idx, listen_row in listening_df.iterrows():
        listen_datetime = pd.to_datetime(listen_row['datetime'])  # Force Timestamp
        artist = listen_row['artist_name']
        track = listen_row['track_name']

        window_start = listen_datetime - timedelta(days=window_days)
        window_end = listen_datetime + timedelta(days=window_days)

        chart_matches = charts_df[
            (charts_df['artist_name'] == artist) &
            (charts_df['track_name'] == track) &
            (charts_df['weekdate'] >= window_start) &
            (charts_df['weekdate'] <= window_end)
        ]

        total_points = chart_matches['weighting'].sum() if not chart_matches.empty else 0

        results.append({
            'datetime': listen_datetime,
            'artist_name': artist,
            'track_name': track,
            'points_awarded': total_points,
            'chart_weeks_matched': len(chart_matches),
            'best_position': chart_matches['position'].min() if not chart_matches.empty else None
        })

    points_df = pd.DataFrame(results)

    # Summary metrics
    total_listens = len(points_df)
    chart_listens = len(points_df[points_df['points_awarded'] > 0])
    total_points = points_df['points_awarded'].sum()
    avg_points = points_df['points_awarded'].mean()
    chart_hit_rate = chart_listens / total_listens if total_listens > 0 else 0

    col1, col2, col3, col4 = st.columns(4)
    with col1:
        st.metric("Total Points", f"{total_points:,.0f}")
    with col2:
        st.metric("Chart Hit Rate", f"{chart_hit_rate:.1%}")
    with col3:
        st.metric("Chart Song Listens", f"{chart_listens:,}")
    with col4:
        st.metric("Avg Points/Listen", f"{avg_points:.1f}")

    # Filter controls
    st.subheader("Filter Results")
    col1, col2, col3 = st.columns(3)
    with col1:
        show_only_hits = st.checkbox("Show only chart hits", value=False)
    with col2:
        min_points = st.number_input("Min points filter", min_value=0, value=0)
    with col3:
        search_artist = st.text_input("Search artist", placeholder="Enter artist name...")

    # Apply filters
    filtered_df = points_df.copy()
    if show_only_hits:
        filtered_df = filtered_df[filtered_df['points_awarded'] > 0]
    if min_points > 0:
        filtered_df = filtered_df[filtered_df['points_awarded'] >= min_points]
    if search_artist:
        filtered_df = filtered_df[filtered_df['artist_name'].str.contains(search_artist, case=False, na=False)]

    # Display results table
    st.subheader("Detailed Results")
    display_df = filtered_df.copy()
    display_df['datetime'] = display_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
    display_df = display_df.rename(columns={
        'datetime': 'Listen Date/Time',
        'artist_name': 'Artist',
        'track_name': 'Track',
        'points_awarded': 'Points',
        'chart_weeks_matched': 'Chart Weeks',
        'best_position': 'Best Position'
    })

    st.dataframe(
        display_df[['Listen Date/Time', 'Artist', 'Track', 'Points', 'Chart Weeks', 'Best Position']],
        use_container_width=True,
        hide_index=True
    )

    st.caption(f"Showing {len(filtered_df):,} of {len(points_df):,} listening instances")

    # Top-performing songs
    chart_hits = points_df[points_df['points_awarded'] > 0]
    if not chart_hits.empty:
        st.subheader("Top Performing Songs")
        top_songs = chart_hits.groupby(['artist_name', 'track_name']).agg({
            'points_awarded': 'sum',
            'chart_weeks_matched': 'mean',
            'datetime': 'count'
        }).reset_index()
        top_songs.columns = ['Artist', 'Track', 'Total Points', 'Avg Chart Weeks', 'Listen Count']
        top_songs = top_songs.sort_values('Total Points', ascending=False).head(10)

        st.dataframe(top_songs, use_container_width=True, hide_index=True)

        # Charts
        col1, col2 = st.columns(2)

        with col1:
            daily_points = chart_hits.copy()
            daily_points['date'] = daily_points['datetime'].dt.date
            daily_summary = daily_points.groupby('date')['points_awarded'].sum().reset_index()

            fig_timeline = px.line(
                daily_summary,
                x='date',
                y='points_awarded',
                title='Points Earned Over Time',
                labels={'points_awarded': 'Points', 'date': 'Date'}
            )
            st.plotly_chart(fig_timeline, use_container_width=True)

        with col2:
            artist_points = chart_hits.groupby('artist_name')['points_awarded'].sum().sort_values(ascending=True).tail(10)
            fig_artists = px.bar(
                x=artist_points.values,
                y=artist_points.index,
                orientation='h',
                title='Top 10 Artists by Points',
                labels={'x': 'Total Points', 'y': 'Artist'}
            )
            st.plotly_chart(fig_artists, use_container_width=True)


TypeError: Invalid comparison between dtype=datetime64[ns] and Timestamp

In [ ]:
listening_df = pd.read_csv("datasets/user_clean/ReRe_20250602_164123.csv")
charts_df = pd.read_csv("datasets/info_clean/info_charts_weighted.csv")

# Clean and convert datetime columns
listening_df["datetime"] = pd.to_datetime(listening_df["datetime"]).dt.tz_localize(None)
charts_df['weekdate'] = pd.to_datetime(charts_df['weekdate'], errors='coerce')
charts_df = charts_df.dropna(subset=['weekdate'])
listening_df['artist_name'] = listening_df['artist_name'].fillna('').str.lower().str.strip()
listening_df['track_name'] = listening_df['track_name'].fillna('').str.lower().str.strip()

# # Sanity check
# st.write("WEEKDATE dtype:", charts_df['weekdate'].dtype)
# st.write("listening date:", listening_df['datetime'].dtype)
# st.write(charts_df['weekdate'].head())
# st.write(listening_df['datetime'].head())

#     listening_df = listening_df.dropna(subset=['datetime'])

window_size = 180
results = []

# Calculate chart points
for idx, listen_row in listening_df.iterrows():
    listen_datetime = listen_row['datetime']
    artist = listen_row['artist_name']
    track = listen_row['track_name']

    # Convert window edges to pandas timestamps
    window_start = pd.Timestamp(listen_datetime - timedelta(days=window_size))
    window_end = pd.Timestamp(listen_datetime)

    chart_matches = charts_df[
        (charts_df['artist_name'] == artist) &
        (charts_df['track_name'] == track) &
        (charts_df['weekdate'] >= window_start) &
        (charts_df['weekdate'] <= window_end)
]

    total_points = chart_matches['weighting'].sum() if not chart_matches.empty else 0

    results.append({
        'datetime': listen_datetime,
        'artist_name': artist,
        'track_name': track,
        'points_awarded': total_points,
        'chart_weeks_matched': len(chart_matches),
        'best_position': chart_matches['position'].min() if not chart_matches.empty else None
    })

points_df = pd.DataFrame(results)

# Summary metrics
total_listens = len(points_df)
chart_listens = len(points_df[points_df['points_awarded'] > 0])
total_points = points_df['points_awarded'].sum()
avg_points = points_df['points_awarded'].mean()
chart_hit_rate = chart_listens / total_listens if total_listens > 0 else 0

# col1, col2, col3, col4 = st.columns(4)
# with col1:
#     st.metric("Total Points", f"{total_points:,.0f}")
# with col2:
#     st.metric("Chart Hit Rate", f"{chart_hit_rate:.1%}")
# with col3:
#     st.metric("Chart Song Listens", f"{chart_listens:,}")
# with col4:
#     st.metric("Avg Points/Listen", f"{avg_points:.1f}")

# Filter controls
# st.subheader("Filter Results")
# col1, col2, col3 = st.columns(3)
# with col1:
#     show_only_hits = st.checkbox("Show only chart hits", value=False)
# with col2:
#     min_points = st.number_input("Min points filter", min_value=0, value=0)
# with col3:
#     search_artist = st.text_input("Search artist", placeholder="Enter artist name...")

# Apply filters
# filtered_df = points_df.copy()
# if show_only_hits:
#     filtered_df = filtered_df[filtered_df['points_awarded'] > 0]
# if min_points > 0:
#     filtered_df = filtered_df[filtered_df['points_awarded'] >= min_points]
# if search_artist:
#     filtered_df = filtered_df[filtered_df['artist_name'].str.contains(search_artist, case=False, na=False)]

# # Display results table
# st.subheader("Detailed Results")
# display_df = filtered_df.copy()
# display_df['datetime'] = display_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
# display_df = display_df.rename(columns={
#     'datetime': 'Listen Date/Time',
#     'artist_name': 'Artist',
#     'track_name': 'Track',
#     'points_awarded': 'Points',
#     'chart_weeks_matched': 'Chart Weeks',
#     'best_position': 'Best Position'
# })

# st.dataframe(
#     display_df[['Listen Date/Time', 'Artist', 'Track', 'Points', 'Chart Weeks', 'Best Position']],
#     use_container_width=True,
#     hide_index=True
# )

# st.caption(f"Showing {len(filtered_df):,} of {len(points_df):,} listening instances")

# # Top-performing songs
# chart_hits = points_df[points_df['points_awarded'] > 0]
# if not chart_hits.empty:
#     st.subheader("Top Performing Songs")
#     top_songs = chart_hits.groupby(['artist_name', 'track_name']).agg({
#         'points_awarded': 'sum',
#         'chart_weeks_matched': 'mean',
#         'datetime': 'count'
#     }).reset_index()
#     top_songs.columns = ['Artist', 'Track', 'Total Points', 'Avg Chart Weeks', 'Listen Count']
#     top_songs = top_songs.sort_values('Total Points', ascending=False).head(10)

#     st.dataframe(top_songs, use_container_width=True, hide_index=True)

#     # Charts
#     col1, col2 = st.columns(2)

#     with col1:
#         daily_points = chart_hits.copy()
#         daily_points['date'] = daily_points['datetime'].dt.date
#         daily_summary = daily_points.groupby('date')['points_awarded'].sum().reset_index()

#         fig_timeline = px.line(
#             daily_summary,
#             x='date',
#             y='points_awarded',
#             title='Points Earned Over Time',
#             labels={'points_awarded': 'Points', 'date': 'Date'}
#         )
#         st.plotly_chart(fig_timeline, use_container_width=True)

#     with col2:
#         artist_points = chart_hits.groupby('artist_name')['points_awarded'].sum().sort_values(ascending=True).tail(10)
#         fig_artists = px.bar(
#             x=artist_points.values,
#             y=artist_points.index,
#             orientation='h',
#             title='Top 10 Artists by Points',
#             labels={'x': 'Total Points', 'y': 'Artist'}
#         )
#         st.plotly_chart(fig_artists, use_container_width=True)

In [56]:
points_df

,datetime,artist_name,track_name,points_awarded,chart_weeks_matched,best_position
0,2025-01-03 14:07:41,Jorge Rivera-Herrans,Mutiny,0,0,None
1,2025-01-03 14:07:41,Jorge Rivera-Herrans,Monster,0,0,None
2,2025-01-03 14:07:41,Jorge Rivera-Herrans,Suffering,0,0,None
3,2025-01-03 14:07:41,Jorge Rivera-Herrans,Scylla,0,0,None
4,2025-01-03 14:07:41,Jorge Rivera-Herrans,Different Beast,0,0,None
...,...,...,...,...,...,...
118397,2020-10-16 10:26:00,Kojo Funds,I Like (feat. WizKid),0,0,None
118398,2020-10-16 10:26:02,Aitch,Rain,0,0,None
118399,2020-10-16 10:26:05,Sneakbo,Love Is A Gamble (feat. Kida Kudz),0,0,None
118400,2020-10-16 10:26:06,Yaadman fka Yung L,Eve Bounce - Remix,0,0,None


In [75]:
print(f'Chart hits listened: {chart_listens}')
print(f'Total points awarded: {total_points}')
print(f'Chart hit rate: {(round(chart_hit_rate,4))}%')

Chart hits listened: 3738
Total points awarded: 802224
Chart hit rate: 0.0316%


In [83]:
# col1, col2, col3, col4 = st.columns(4)
# with col1:
#     st.metric("Total Points", f"{total_points:,.0f}")
# with col2:
#     st.metric("Chart Hit Rate", f"{chart_hit_rate:.1%}")
# with col3:
#     st.metric("Chart Song Listens", f"{chart_listens:,}")
# with col4:
#     st.metric("Avg Points/Listen", f"{avg_points:.1f}")

# Filter controls
# st.subheader("Filter Results")
# col1, col2, col3 = st.columns(3)
# with col1:
#     show_only_hits = st.checkbox("Show only chart hits", value=False)
# with col2:
#     min_points = st.number_input("Min points filter", min_value=0, value=0)
# with col3:
#     search_artist = st.text_input("Search artist", placeholder="Enter artist name...")

# Apply filters
# filtered_df = points_df.copy()
# if show_only_hits:
#     filtered_df = filtered_df[filtered_df['points_awarded'] > 0]
# if min_points > 0:
#     filtered_df = filtered_df[filtered_df['points_awarded'] >= min_points]
# if search_artist:
#     filtered_df = filtered_df[filtered_df['artist_name'].str.contains(search_artist, case=False, na=False)]

# Display results table
# display_df = filtered_df.copy()
display_df = points_df.copy()
display_df['datetime'] = display_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
display_df = display_df.rename(columns={
    'datetime': 'Listen Date/Time',
    'artist_name': 'Artist',
    'track_name': 'Track',
    'points_awarded': 'Points',
    'chart_weeks_matched': 'Chart Weeks',
    'best_position': 'Best Position'
})

results_table = pd.DataFrame(
    display_df[['Listen Date/Time', 'Artist', 'Track', 'Points', 'Chart Weeks', 'Best Position']],
)

# st.caption(f"Showing {len(filtered_df):,} of {len(points_df):,} listening instances")

# # Top-performing songs
chart_hits = points_df[points_df['points_awarded'] > 0]
if not chart_hits.empty:
    # st.subheader("Top Performing Songs")
    top_songs = chart_hits.groupby(['artist_name', 'track_name']).agg({
        'points_awarded': 'sum',
        'chart_weeks_matched': 'mean',
        'datetime': 'count'
    }).reset_index()
    top_songs.columns = ['Artist', 'Track', 'Total Points', 'Avg Chart Weeks', 'Listen Count']
    top_songs = top_songs.sort_values('Total Points', ascending=False).head(10)

top_performing_songs = pd.DataFrame(top_songs)

#     # Charts
#     col1, col2 = st.columns(2)

#     with col1:
daily_points = chart_hits.copy()
daily_points['date'] = daily_points['datetime'].dt.date
daily_summary = daily_points.groupby('date')['points_awarded'].sum().reset_index()

# fig_timeline =
px.line(
    daily_summary,
    x='date',
    y='points_awarded',
    title='Points Earned Over Time',
    labels={'points_awarded': 'Points', 'date': 'Date'}
)
# st.plotly_chart(fig_timeline, use_container_width=True)

#     with col2:
#         artist_points = chart_hits.groupby('artist_name')['points_awarded'].sum().sort_values(ascending=True).tail(10)
#         fig_artists = px.bar(
#             x=artist_points.values,
#             y=artist_points.index,
#             orientation='h',
#             title='Top 10 Artists by Points',
#             labels={'x': 'Total Points', 'y': 'Artist'}
#         )
#         st.plotly_chart(fig_artists, use_container_width=True)

In [86]:
artist_points = chart_hits.groupby('artist_name')['points_awarded'].sum().sort_values(ascending=True).tail(10)
px.bar(
            x=artist_points.values,
            y=artist_points.index,
            orientation='h',
            title='Top 10 Artists by Points',
            labels={'x': 'Total Points', 'y': 'Artist'})

In [ ]:
# Full set of window sizes

listening_df = pd.read_csv("datasets/user_clean/ReRe_20250602_164123.csv")
charts_df = pd.read_csv("datasets/info_clean/info_charts_weighted.csv")

# Clean and convert datetime columns
listening_df["datetime"] = pd.to_datetime(listening_df["datetime"]).dt.tz_localize(None)
charts_df['weekdate'] = pd.to_datetime(charts_df['weekdate'], errors='coerce')
charts_df = charts_df.dropna(subset=['weekdate'])
listening_df['artist_name'] = listening_df['artist_name'].fillna('').str.lower().str.strip()
listening_df['track_name'] = listening_df['track_name'].fillna('').str.lower().str.strip()

# Window sizes
window_size = [365, 182, 91, 61, 30, 7]

# To store all results
all_points_dfs = {}
summary_stats = {}

for w in window_size:
    results = []

    for idx, listen_row in listening_df.iterrows():
        listen_datetime = listen_row['datetime']
        artist = listen_row['artist_name']
        track = listen_row['track_name']

        window_start = pd.Timestamp(listen_datetime - timedelta(days=w))
        window_end = pd.Timestamp(listen_datetime)

        chart_matches = charts_df[
            (charts_df['artist_name'] == artist) &
            (charts_df['track_name'] == track) &
            (charts_df['weekdate'] >= window_start) &
            (charts_df['weekdate'] <= window_end)
        ]

        total_points = chart_matches['weighting'].sum() if not chart_matches.empty else 0

        results.append({
            'datetime': listen_datetime,
            'artist_name': artist,
            'track_name': track,
            'points_awarded': total_points,
            'chart_weeks_matched': len(chart_matches),
            'best_position': chart_matches['position'].min() if not chart_matches.empty else None
        })

    # Big old results dataframe
    df = pd.DataFrame(results)
    all_points_dfs[f'points_df_{w}'] = df

    # Agg stats
    total_listens = len(df)
    chart_listens = len(df[df['points_awarded'] > 0])
    total_points = df['points_awarded'].sum()
    avg_points = df['points_awarded'].mean()
    chart_hit_rate = chart_listens / total_listens if total_listens > 0 else 0

    summary_stats[f'summary_{w}'] = {
        'total_listens': total_listens,
        'chart_listens': chart_listens,
        'total_points': total_points,
        'avg_points': avg_points,
        'chart_hit_rate': chart_hit_rate
    }


In [91]:
# look at the points for 30 day window
points_30 = all_points_dfs['points_df_30']
summary_30 = summary_stats['summary_7']
print(summary_30)

{'total_listens': 118402, 'chart_listens': 1953, 'total_points': 65578, 'avg_points': 0.5538588875187919, 'chart_hit_rate': 0.016494653806523537}


In [ ]:
all_points_dfs.to_json("datasets/chart_scores/ReRe_20250602_164123_all_points")
summary_stats.to_json("datasets/chart_scores/ReRe_20250602_164123_summary_stats")

AttributeError: 'dict' object has no attribute 'to_json'

In [109]:
with open("datasets/chart_scores/ReRe_20250602_164123_summary_stats.json", 'w') as to_json:
    json.dump(summary_stats, to_json)

TypeError: Object of type int64 is not JSON serializable

In [107]:
all_points_dfs

{'points_df_365':                   datetime           artist_name  \
 0      2025-01-03 14:07:41  jorge rivera-herrans   
 1      2025-01-03 14:07:41  jorge rivera-herrans   
 2      2025-01-03 14:07:41  jorge rivera-herrans   
 3      2025-01-03 14:07:41  jorge rivera-herrans   
 4      2025-01-03 14:07:41  jorge rivera-herrans   
 ...                    ...                   ...   
 118397 2020-10-16 10:26:00            kojo funds   
 118398 2020-10-16 10:26:02                 aitch   
 118399 2020-10-16 10:26:05               sneakbo   
 118400 2020-10-16 10:26:06    yaadman fka yung l   
 118401 2020-10-16 10:26:40              omah lay   
 
                                 track_name  points_awarded  \
 0                                   mutiny               0   
 1                                  monster               0   
 2                                suffering               0   
 3                                   scylla               0   
 4                          di

In [108]:
type(all_points_dfs)

dict

In [115]:
import os
from pathlib import Path

In [ ]:
import os
from pathlib import Path
file_path = Path("datasets/user_clean/ReRe_20250602_164123.csv")
file_stem = file_path.stem
file_stem

'ReRe_20250602_164123'

In [127]:
summary_stats

{'summary_365': {'total_listens': 118402,
  'chart_listens': 4681,
  'total_points': 1215483,
  'avg_points': 10.265730308609651,
  'chart_hit_rate': 0.03953480515531832},
 'summary_182': {'total_listens': 118402,
  'chart_listens': 3750,
  'total_points': 808153,
  'avg_points': 6.825501258424689,
  'chart_hit_rate': 0.03167176230131248},
 'summary_91': {'total_listens': 118402,
  'chart_listens': 3070,
  'total_points': 525670,
  'avg_points': 4.439705410381581,
  'chart_hit_rate': 0.02592861607067448},
 'summary_61': {'total_listens': 118402,
  'chart_listens': 2713,
  'total_points': 407421,
  'avg_points': 3.440997618283475,
  'chart_hit_rate': 0.022913464299589535},
 'summary_30': {'total_listens': 118402,
  'chart_listens': 2413,
  'total_points': 241041,
  'avg_points': 2.035784868498843,
  'chart_hit_rate': 0.020379723315484537},
 'summary_7': {'total_listens': 118402,
  'chart_listens': 1953,
  'total_points': 65578,
  'avg_points': 0.5538588875187919,
  'chart_hit_rate': 0.0

In [129]:
all_points_dict = {k: v.to_dict(orient='records') for k, v in all_points_dfs.items()}
with open("datasets/chart_scores/ReRe_20250602_164123_all_points.json", "w") as f:
    json.dump(all_points_dict, f)

TypeError: Object of type Timestamp is not JSON serializable

In [131]:
import pickle

with open("datasets/chart_scores/ReRe_20250602_164123_all_points.pkl", "wb") as f:
    pickle.dump(all_points_dfs, f)

# # To load later
# with open("all_points_dfs.pkl", "rb") as f:
#     all_points_dfs = pickle.load(f)


In [132]:
with open(f"datasets/chart_scores/ReRe_20250602_164123_summary_stats.pkl", "wb") as f:
    pickle.dump(summary_stats, f)

In [133]:
# To load later
with open("datasets/chart_scores/ReRe_20250602_164123_all_points.pkl", "rb") as f:
    check_ap = pickle.load(f)
check_ap


{'points_df_365':                   datetime           artist_name  \
 0      2025-01-03 14:07:41  jorge rivera-herrans   
 1      2025-01-03 14:07:41  jorge rivera-herrans   
 2      2025-01-03 14:07:41  jorge rivera-herrans   
 3      2025-01-03 14:07:41  jorge rivera-herrans   
 4      2025-01-03 14:07:41  jorge rivera-herrans   
 ...                    ...                   ...   
 118397 2020-10-16 10:26:00            kojo funds   
 118398 2020-10-16 10:26:02                 aitch   
 118399 2020-10-16 10:26:05               sneakbo   
 118400 2020-10-16 10:26:06    yaadman fka yung l   
 118401 2020-10-16 10:26:40              omah lay   
 
                                 track_name  points_awarded  \
 0                                   mutiny               0   
 1                                  monster               0   
 2                                suffering               0   
 3                                   scylla               0   
 4                          di